## PUC Minas
### Pós-Graduação em Ciência de Dados e Big Data
#### Avaliação Final - Modelagem e Preparação de Dados para Aprendizado de Máquina

**Aluno(s):**

In [62]:
# Luciano Augusto Scherer Contri

### Base de Dados

**Descriçao de Atributos**

- age: idade
- workclass: classe de trabalho
- education: nível educacional
- education-num: anos de educação
- marital-status: estado civil
- occupation: profissão
- race: etnia
- sex: gênero
- capital-gain: ganho de capital
- capital-loss: perda de capital
- hours-per-week: horas de trabalho por semana
- native-country: país de origem

**Contexto dos Dados**

O dataset apresenta dados de um problema de classificação onde o objetivo é prever se a pessoa da observação ganha mais de 50k dólares por ano ou não.

## Atividades

**Dados**

In [63]:
# Carregue o dataset fornecido ('adult_final.csv')

In [64]:
import pandas as pd
        
adult_final = pd.read_csv('../../Datasets/adult_final.csv', sep = ',')
adult_final

**1. Apresente o tipo das variáveis.**

In [65]:
adult_final.dtypes

**2.Apresente de forma gráfica e numérica a análise exploratória das variáveis _education_ e _race_.**

In [66]:
adult_final['education'].value_counts().plot(kind='bar')

In [67]:
# porcentagem 
adult_final['education'].value_counts(normalize=True) * 100

In [68]:
adult_final['race'].value_counts().plot(kind='bar')

In [69]:
# porcentagem
adult_final['race'].value_counts(normalize=True) * 100

**3. Apresente as métricas estatísticas (média, moda, etc.) e histograma das variáveis _age_ e _hours-per-week_.**

In [70]:
# agregações
adult_final.agg({'age': ['mean', 'median', 'std', 'var', 'min', 'max'], 'hours-per-week': ['mean', 'median', 'std', 'var', 'min', 'max']})

**4. Apresente 2 análises multivaridas entre variáveis a sua escolha.**

In [71]:
# média de idade por classe de trabalho
adult_final.groupby('workclass')[['capital-gain', 'age']].mean().sort_values(by='age')

In [72]:
# média de horas por semana por classe de trabalho
adult_final.groupby('workclass')['hours-per-week'].mean().sort_values()

In [73]:
# média de horas por semana por ocupação
adult_final.groupby('occupation')['hours-per-week'].mean().sort_values()

In [74]:
# workclass por média de idade
adult_final.groupby('workclass')['age'].mean().sort_values()

In [75]:
# média de education-num por target
adult_final.groupby('workclass')['education-num'].mean().sort_values()

**5. Apresente a soma de _NaN_ de cada coluna da base de dados.**

In [76]:
adult_final.isna().sum()

**6. Trate os _NaN_ de todas as colunas como achar conveniente (explique). Em seguida, mostre que nenhuma coluna apresenta _NaN_ ao final do processo.**

Como as colunas são categoricas, não podemos substituir os valores faltantes pela média ou mediana, pois não faz sentido.   
podemos substituir os valores faltantes pela moda, ou seja, o valor mais frequente. 
Mas isso pode enviesar o modelo, pois a moda pode não ser a melhor escolha para substituir os valores faltantes.
Outra opção é usar o KNNImputer, que é um método de imputação que utiliza os k vizinhos mais próximos para preencher os valores faltantes.

### KNN

In [144]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import NearestNeighbors

from sklearn.metrics import accuracy_score, f1_score, balanced_accuracy_score

# Carregar e preparar o DataFrame
df = adult_final.copy()
num_cols = ['education-num', 'age', 'capital-gain', 'hours-per-week']
cat_cols = ['workclass', 'occupation', 'native-country']

# hashmap para workclass em ordem crescente
workclass_map = {'Federal-gov': 1, 'Local-gov': 2, 'State-gov': 3, 'Self-emp-inc': 4, 'Self-emp-not-inc': 5, 'Private': 6, 'Without-pay': 7, 'Never-worked': 8}

occupation_map = {'Prof-specialty': 1, 'Exec-managerial': 2, 'Protective-serv': 3, 'Tech-support': 4, 'Sales': 5, 'Craft-repair': 6, 'Transport-moving': 7, 'Adm-clerical': 8, 'Farming-fishing': 9, 'Machine-op-inspct': 10, 'Handlers-cleaners': 11, 'Other-service': 12, 'Armed-Forces': 13, 'Priv-house-serv': 14}

native_country_map = {'United-States': 1, 'Mexico': 2, 'Philippines': 3, 'Germany': 4, 'Puerto-Rico': 5, 'Canada': 6, 'El-Salvador': 7, 'India': 8, 'Cuba': 9, 'England': 10, 'Jamaica': 11, 'South': 12, 'China': 13, 'Italy': 14, 'Dominican-Republic': 15, 'Vietnam': 16, 'Guatemala': 17, 'Japan': 18, 'Poland': 19, 'Columbia': 20, 'Taiwan': 21, 'Haiti': 22, 'Iran': 23, 'Portugal': 24, 'Nicaragua': 25, 'Peru': 26, 'Greece': 27, 'Ecuador': 28, 'France': 29, 'Ireland': 30, 'Hong': 31, 'Cambodia': 32, 'Trinadad&Tobago': 33, 'Thailand': 34, 'Laos': 35, 'Yugoslavia': 36, 'Outlying-US(Guam-USVI-etc)': 37, 'Honduras': 38, 'Hungary': 39, 'Scotland': 40, 'Holand-Netherlands': 41}

# transformar workclass em ordem crescente
df['workclass'] = df['workclass'].map(workclass_map)
df['occupation'] = df['occupation'].map(occupation_map)
df['native-country'] = df['native-country'].map(native_country_map)

for i in cat_cols:
    # Separe os dados em conjunto de treinamento e teste
    train_df, test_df = train_test_split(df[num_cols + cat_cols].dropna(), test_size=0.2, random_state=42)
    
    # Treine um modelo de classificação (por exemplo, KNN) usando os dados de treinamento
    knn_classifier = KNeighborsClassifier(n_neighbors=5)
    knn_classifier.fit(train_df.drop(columns=[i]), train_df[i])  # 'workclass' é sua coluna alvo
    
    # Avalie a acurácia do modelo usando os dados de teste imputados
    y_true = test_df[i]  # 'workclass' é sua coluna alvo
    y_pred = knn_classifier.predict(test_df.drop(columns=[i]))  # 'workclass' é sua coluna alvo
    # Calcular balanced_accuracy_score
    
    accuracy = balanced_accuracy_score(y_true, y_pred)
    
    print(f"Acurácia do modelo imputado {i}: {accuracy:.3f}")

Acurácia do modelo imputado workclass: 0.692
Acurácia do modelo imputado occupation: 0.227
Acurácia do modelo imputado native-country: 0.918

Tendo em vista que os dados faltantes são menores que 10% do total, podemos usar o KNNImputer para preencher os valores faltantes.

In [131]:
# comparing the original and imputed values to see how many changed with a sum
(df[cat_cols] != adult_final[cat_cols]).sum()

In [132]:
df['workclass'].unique()

Ta perfeito demais pra ser verdade, mas pode ser apenas porquê o dataset é pequeno. de qualquer forma, vamos imputar os valores faltantes no dataset original.

In [86]:
# imputar no dataset original   
df_imputed = adult_final.copy()
df_imputed[num_cols + cat_cols] = imputer.fit_transform(df[num_cols + cat_cols])
# Decodificar as colunas categóricas imputadas para os valores originais
for col in cat_cols:
    df_imputed[col] = encoders[col].inverse_transform(df_imputed[col].astype(int))


In [126]:
df.isna().sum()

In [122]:
df[['workclass', 'occupation', 'native-country']].eq('Desconhecido').sum()


In [20]:
# Decodificar as colunas categóricas imputadas para os valores originais
for col in cat_cols:
    df_imputed[col] = encoders[col].inverse_transform(df_imputed[col].astype(int))

# Verificar se a decodificação foi bem-sucedida
df_imputed

In [21]:
# tabela unica de nunique() onde index é o nome da coluna e as colunas são os valores únicos de cada dataframe
pd.concat([adult_final.nunique(), df_imputed.nunique()], axis=1, keys=['adult_final', 'df_imputed'])

Podemos ver que o número de valores únicos aumentou em 1 pois o encoder adicionou um valor para os valores faltantes. como demonstrado abaixo.

In [22]:
# adult_final.nunique() e df_imputed.nunique() para verificar se o número de valores únicos é o mesmo, lado a lado
print(adult_final['workclass'].unique()) 
print(adult_final['workclass'].nunique())
print(df_imputed['workclass'].unique())
print(df_imputed['workclass'].nunique())

**7. Aplique _Ordinal Encoding_ em uma variável categórica ordinal.**

In [23]:
# ordinal encoding na coluna 'education'
from sklearn.preprocessing import OrdinalEncoder


df_imputed['education'] = OrdinalEncoder().fit_transform(df_imputed[['education']])
df_imputed['education']

**8. Aplique _One Hot Encoding_ em uma variável categórica nominal.**

In [24]:
# categorias nominais: workclass, occupation e native-country

# one hot encoding na coluna 'workclass'
df_imputed = pd.get_dummies(df_imputed, columns=['workclass']) 

In [29]:
df_imputed['workclass_nan']

**9. Aplique uma técnica de _oversampling_ (classe minoritária) e uma de _undersampling_ (classe majoritária). Apresente a mudança de volumetria (antes e depois). Se necessário, lembre-se de tratar as variáveis categóricas de forma adequada caso deseje usar um método mais robusto (SMOTE, por exemplo). Se for o caso, utilize PCA para visualizar os dados de forma bidimensional (antes e depois da amostragem).**

**10. Aplique _One Hot Encoding_ nas variáveis _race_ e _sex_. Junte ao resultado _TODAS_ as outras variáveis númericas (_age_, _education-num_, _capital-gain_, _capital-loss_ e _hours-per-week_). Utilize o dataset resultante no algoritmo t-SNE e reduza a dimensionalidade à 2 componentes (padrão do algoritmo). Plote o resultado diferenciando os pontos pela classe (atributo _target_).**